# Projeto de Bases de Dados - Parte 3

### Docente Responsável

Prof. André Patrício
Prof. Jorge Oliveira

### Grupo 4 - BD2L17
<dl>
    <dt>40 horas (33.3%)</dt>
    <dd>ist102447 Guilherme Belchior</dd>
    <dt>40 horas (33.3%)</dt>
    <dd>ist103095 Daniel Nunes</dd>
    <dt>40 horas (33.3%)</dt>
    <dd>ist102078 João Costa</dd>
<dl>

In [ ]:
%load_ext sql
%sql postgresql://db:db@postgres/db



# Empresa de comércio online

## 0. Carregamento da Base de Dados

Carregue o esquema de Base de Dados apresentado no Anexo A.

In [ ]:
%%sql

DROP TABLE IF EXISTS customer CASCADE;
DROP TABLE IF EXISTS orders CASCADE;
DROP TABLE IF EXISTS pay CASCADE;
DROP TABLE IF EXISTS employee CASCADE;
DROP TABLE IF EXISTS process CASCADE;
DROP TABLE IF EXISTS department CASCADE;
DROP TABLE IF EXISTS workplace CASCADE;
DROP TABLE IF EXISTS works CASCADE;
DROP TABLE IF EXISTS office CASCADE;
DROP TABLE IF EXISTS warehouse CASCADE;
DROP TABLE IF EXISTS product CASCADE;
DROP TABLE IF EXISTS contains CASCADE;
DROP TABLE IF EXISTS supplier CASCADE;
DROP TABLE IF EXISTS delivery CASCADE;

/* Change this above */



CREATE TABLE customer(
  cust_no INTEGER PRIMARY KEY,
  name VARCHAR(80) NOT NULL,
  email VARCHAR(254) UNIQUE NOT NULL,
  phone VARCHAR(15),
  address VARCHAR(255)
);

CREATE TABLE orders(
  order_no INTEGER PRIMARY KEY,
  cust_no INTEGER NOT NULL REFERENCES customer,
  date DATE NOT NULL
  --order_no must exist in contains
);

CREATE TABLE pay(
  order_no INTEGER PRIMARY KEY REFERENCES orders,
  cust_no INTEGER NOT NULL REFERENCES customer
);

CREATE TABLE employee(
  ssn VARCHAR(20) PRIMARY KEY,
  TIN VARCHAR(20) UNIQUE NOT NULL,
  bdate DATE,
  name VARCHAR NOT NULL
  --age must be >=18
);

CREATE TABLE process(
  ssn VARCHAR(20) REFERENCES employee,
  order_no INTEGER REFERENCES orders,
  PRIMARY KEY (ssn, order_no)
);

CREATE TABLE department(
  name VARCHAR PRIMARY KEY
);

CREATE TABLE workplace(
  address VARCHAR PRIMARY KEY,
  lat NUMERIC(8, 6) NOT NULL,
  long NUMERIC(9, 6) NOT NULL,
  UNIQUE(lat, long)
  --address must be in warehouse or office but not both
);

CREATE TABLE office(
  address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE warehouse(
  address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE works(
  ssn VARCHAR(20) REFERENCES employee,
  name VARCHAR(200) REFERENCES department,
  address VARCHAR(255) REFERENCES workplace,
  PRIMARY KEY (ssn, name, address)
);

CREATE TABLE product(
  SKU VARCHAR(25) PRIMARY KEY,
  name VARCHAR(200) NOT NULL,
  description VARCHAR,
  price NUMERIC(10, 2) NOT NULL,
  ean NUMERIC(13) UNIQUE
);

CREATE TABLE contains(
  order_no INTEGER REFERENCES orders,
  SKU VARCHAR(25) REFERENCES product,
  qty INTEGER,
  PRIMARY KEY (order_no, SKU)
);

CREATE TABLE supplier(
  TIN VARCHAR(20) PRIMARY KEY,
  name VARCHAR(200),
  address VARCHAR(255),
  SKU VARCHAR(25) REFERENCES product,
  date DATE
);

CREATE TABLE delivery(
  address VARCHAR(255) REFERENCES warehouse ,
  TIN VARCHAR(20) REFERENCES supplier,
  PRIMARY KEY (address, TIN)
);

Crie as instruções para o seu preenchimento de forma consistente, garantindo que todas as consultas SQL e OLAP, apresentadas mais adiante, produzam um resultado não vazio. 

In [ ]:
%%sql
insert into customer (cust_no, name, email, phone, address) VALUES (-1,'Deleted_User', '', '', '');
insert into customer (cust_no, name, email, phone, address) VALUES (1,'Belchior', 'belchiorlindo@gmail.com', '912345678', 'Avenida da Liberdade, 2000-002 Póvoa de Varzim');
insert into customer (cust_no, name, email, phone, address) VALUES (2,'Nunes', 'danielnunes@gmail.com', '939393939', 'Travessa dos Pescadores, 3000-003 Coimbra');
insert into customer (cust_no, name, email, phone, address) VALUES (3,'Costa', 'joaolscosta@gmail.com', '947854684', 'Rua das Flores, 1000-001 Lisboa');


insert into orders (order_no, date, cust_no) VALUES (1,'2017-01-01', (select cust_no from customer where name = 'Belchior'));
insert into orders (order_no, date, cust_no) VALUES (2,'2018-01-01', (select cust_no from customer where name = 'Nunes'));
insert into orders (order_no, date, cust_no) VALUES (3,'2019-01-01', (select cust_no from customer where name = 'Costa'));
insert into orders (order_no, date, cust_no) VALUES (4,'2014-01-01', (select cust_no from customer where name = 'Costa'));
insert into orders (order_no, date, cust_no) VALUES (5,'2023-01-02', (select cust_no from customer where name = 'Costa'));
insert into orders (order_no, date, cust_no) VALUES (6,'2023-02-02', (select cust_no from customer where name = 'Nunes'));
insert into orders (order_no, date, cust_no) VALUES (7,'2023-01-03', (select cust_no from customer where name = 'Nunes'));
insert into orders (order_no, date, cust_no) VALUES (8,'2022-01-03', (select cust_no from customer where name = 'Nunes'));
insert into orders (order_no, date, cust_no) VALUES (9,'2022-01-04', (select cust_no from customer where name = 'Nunes'));
insert into orders (order_no, date, cust_no) VALUES (10,'2022-01-04', (select cust_no from customer where name = 'Nunes'));
insert into orders (order_no, date, cust_no) VALUES (11,'2022-03-04', (select cust_no from customer where name = 'Nunes'));
insert into orders (order_no, date, cust_no) VALUES (12,'2022-10-04', (select cust_no from customer where name = 'Nunes'));




insert into pay (order_no, cust_no) VALUES ((select order_no from orders where date = '2017-01-01'), (select cust_no from customer where name = 'Belchior'));
insert into pay (order_no, cust_no) VALUES ((select order_no from orders where date = '2018-01-01'), (select cust_no from customer where name = 'Nunes'));
insert into pay (order_no, cust_no) VALUES (8, 2);
insert into pay (order_no, cust_no) VALUES (9, 2);

insert into employee (ssn, TIN, bdate, name) VALUES ('546654654', '123123123', '2002-04-10', 'Pedrinho');
insert into employee (ssn, TIN, bdate, name) VALUES ('546654655', '123123124', '2002-02-14', 'Gui');
insert into employee (ssn, TIN, bdate, name) VALUES ('546654656', '123123125', '2002-01-01', 'Luana');

insert into process (ssn, order_no) VALUES ((select ssn from employee where name = 'Pedrinho'), (select order_no from orders where date = '2017-01-01'));
insert into process (ssn, order_no) VALUES ((select ssn from employee where name = 'Luana'), (select order_no from orders where date = '2023-01-02'));
insert into process (ssn, order_no) VALUES ((select ssn from employee where name = 'Gui'), (select order_no from orders where date = '2019-01-01'));
insert into process (ssn, order_no) VALUES ((select ssn from employee where name = 'Gui'), (select order_no from orders where date = '2023-01-03'));
insert into process (ssn, order_no) VALUES ((select ssn from employee where name = 'Pedrinho'), (select order_no from orders where date = '2014-01-01'));
insert into process (ssn, order_no) VALUES ((select ssn from employee where name = 'Gui'), 11);
insert into process (ssn, order_no) VALUES ((select ssn from employee where name = 'Gui'), 12);


insert into department (name) VALUES ('Marketing');
insert into department (name) VALUES ('Production');

insert into workplace (address, lat, long) VALUES ('Rua das Flores, 1000-001 Lisboa', '12.000000', '45.000000');
insert into workplace (address, lat, long) VALUES ('Rua do Comércio, 4000-004 Porto', '78.000000', '10.000000');
insert into workplace (address, lat, long) VALUES ('Praça da República, 5000-005 Vila Real', '78.000000', '13.000000');
insert into workplace (address, lat, long) VALUES ('Avenida Central, 6000-006 Castelo Branco', '78.000000', '11.000000');


insert into office (address) VALUES ((select address from workplace where address = 'Praça da República, 5000-005 Vila Real'));
insert into office (address) VALUES ((select address from workplace where address = 'Avenida Central, 6000-006 Castelo Branco'));

insert into warehouse (address) VALUES ((select address from workplace where address = 'Rua das Flores, 1000-001 Lisboa'));
insert into warehouse (address) VALUES ((select address from workplace where address = 'Rua do Comércio, 4000-004 Porto'));


insert into works (ssn, name, address) VALUES ((select ssn from employee where name = 'Pedrinho'), (select name from department where name = 'Marketing'), (select address from workplace where address = 'Praça da República, 5000-005 Vila Real'));
insert into works (ssn, name, address) VALUES ((select ssn from employee where name = 'Luana'), (select name from department where name = 'Marketing'), (select address from workplace where address = 'Rua das Flores, 1000-001 Lisboa'));
insert into works (ssn, name, address) VALUES ((select ssn from employee where name = 'Gui'), (select name from department where name = 'Production'), (select address from workplace where address = 'Rua das Flores, 1000-001 Lisboa'));
insert into works (ssn, name, address) VALUES ((select ssn from employee where name = 'Gui'), (select name from department where name = 'Production'), (select address from workplace where address = 'Avenida Central, 6000-006 Castelo Branco'));


insert into product (SKU, name, description, price, ean) VALUES ('-1', '', '',0, '-1');
insert into product (SKU, name, description, price, ean) VALUES ('EI01TS01EI01TS01EI01TS011', 'Gaming Chair', 'Chair made with love and kindness', 5.35, '1234567891234');
insert into product (SKU, name, description, price, ean) VALUES ('EI01TS01EI01TS01EI01TS012', 'Cake', 'Cake made by our amazing chef Luana', 10.35, '1234567891235');
insert into product (SKU, name, description, price, ean) VALUES ('EI01TS01EI01TS01EI01TS013', 'TV', 'LG TV 4K', 500.35, '1234567891236');
insert into product (SKU, name, description, price, ean) VALUES ('EI01TS01EI01TS01EI01TS014', 'Table', 'Table made with paus and cola', 50.35, '1234567891237');



insert into contains (SKU, order_no, qty) VALUES ((select SKU from product where name = 'Gaming Chair'), (select order_no from orders where date = '2017-01-01'), 2);
insert into contains (SKU, order_no, qty) VALUES ((select SKU from product where name = 'Cake'), (select order_no from orders where date = '2018-01-01'), 10);
insert into contains (SKU, order_no, qty) VALUES ((select SKU from product where name = 'TV'), (select order_no from orders where date = '2019-01-01'), 3);
insert into contains (SKU, order_no, qty) VALUES ((select SKU from product where name = 'TV'), (select order_no from orders where date = '2023-01-02'), 14);
insert into contains (SKU, order_no, qty) VALUES ((select SKU from product where name = 'TV'), (select order_no from orders where date = '2023-01-03'), 3);
insert into contains (SKU, order_no, qty) VALUES ((select SKU from product where name = 'TV'), 8, 14);
insert into contains (SKU, order_no, qty) VALUES ((select SKU from product where name = 'TV'), 9, 20);
insert into contains (SKU, order_no, qty) VALUES ((select SKU from product where name = 'TV'), 10, 2);
insert into contains (SKU, order_no, qty) VALUES ((select SKU from product where name = 'TV'), 11, 2);
insert into contains (SKU, order_no, qty) VALUES ((select SKU from product where name = 'TV'), 12, 2);
insert into contains (SKU, order_no, qty) VALUES ((select SKU from product where name = 'TV'), 4, 14);
insert into contains (SKU, order_no, qty) VALUES ((select SKU from product where name = 'TV'), 6, 14);


insert into supplier (TIN, name, address, SKU, date) VALUES ('123456789', 'Goncalo', 'Rua das Palmeiras, 7000-007 Évora', (select SKU from product where name = 'Gaming Chair'), '2017-01-01');
insert into supplier (TIN, name, address, SKU, date) VALUES ('123456788', 'Rafael', 'Avenida do Mar, 8000-008 Faro', (select SKU from product where name = 'Cake'), '2018-01-01');
insert into supplier (TIN, name, address, SKU, date) VALUES ('123456787', 'Luana', 'Travessa das Oliveiras, 9000-009 Funchal', (select SKU from product where name = 'TV'), '2019-01-01');
insert into supplier (TIN, name, address, SKU, date) VALUES ('123456786', 'Ricky', 'Rua dos Castanheiros, 1000-010 Guarda', (select SKU from product where name = 'TV'), '2023-01-02');



insert into delivery (address, TIN) VALUES ((select address from warehouse where address = 'Rua das Flores, 1000-001 Lisboa'), (select TIN from supplier where name = 'Goncalo'));
insert into delivery (address, TIN) VALUES ((select address from warehouse where address = 'Rua do Comércio, 4000-004 Porto'), (select TIN from supplier where name = 'Luana'));
insert into delivery (address, TIN) VALUES ((select address from warehouse where address = 'Rua das Flores, 1000-001 Lisboa'), (select TIN from supplier where name = 'Rafael'));
insert into delivery (address, TIN) VALUES ((select address from warehouse where address = 'Rua do Comércio, 4000-004 Porto'), (select TIN from supplier where name = 'Ricky'));

## 1. Restrições de Integridade

Apresente o código para implementar as seguintes restrições de integridade, se necessário, com recurso a extensões procedimentais SQL (Stored Procedures e Triggers):

(RI-1) Nenhum empregado pode ter menos de 18 anos de idade

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION ValidAge() RETURNS TRIGGER AS
$$
  BEGIN
    IF ((CURRENT_DATE - 6570) < NEW.bdate ) THEN
      RAISE EXCEPTION 'Employees must be 18 years old';
    END IF;
    RETURN NEW;
  END
$$ LANGUAGE plpgsql;

CREATE TRIGGER CheckValidAge
BEFORE INSERT OR UPDATE ON employee
FOR EACH ROW EXECUTE PROCEDURE ValidAge();

(RI-2) Um 'Workplace' é obrigatoriamente um 'Office' ou 'Warehouse' mas não pode ser ambos

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION CheckValidWorkplace() RETURNS TRIGGER AS
$$
  BEGIN
    IF (NEW.address NOT IN (SELECT address FROM Office) AND
        NEW.address NOT IN (SELECT address FROM Warehouse)) 
      OR
       ( NEW.address IN (SELECT address FROM Office) AND
         NEW.address IN (SELECT address FROM Warehouse))  
      THEN
      RAISE EXCEPTION 'Workplace must be either an Office or a Warehouse and not Both!';
    END IF;
    RETURN NEW;
  END
$$ LANGUAGE plpgsql;

CREATE TRIGGER CheckValidWorkplace
BEFORE INSERT OR UPDATE ON Workplace
FOR EACH ROW EXECUTE FUNCTION CheckValidWorkplace();


(RI-3) Uma 'Order' tem de figurar obrigatoriamente em 'Contains'.

In [1]:
%%sql
CREATE OR REPLACE FUNCTION CheckValidOrder() RETURNS TRIGGER AS
$$
  BEGIN
    IF (NEW.order_no NOT IN (SELECT order_no FROM contains)) THEN
      RAISE EXCEPTION 'Orders must have at least one product!';
    END IF;
    RETURN NEW;
  END
$$ LANGUAGE plpgsql;

CREATE CONSTRAINT TRIGGER CheckValidOrder
AFTER INSERT OR UPDATE ON orders 
DEFERRABLE 
INITIALLY DEFERRED
FOR EACH ROW EXECUTE PROCEDURE CheckValidOrder();

UsageError: Cell magic `%%sql` not found.


## 2. Consultas SQL

Apresente a consulta SQL mais sucinta para cada uma das seguintes questões

1) Qual o número e nome do(s) cliente(s) com maior valor total de encomendas pagas?  

In [ ]:
%%sql
SELECT c.name, c.cust_no
FROM customer c
JOIN pay p ON c.cust_no = p.cust_no
GROUP BY c.cust_no, name HAVING COUNT(*) >= ALL (
    SELECT COUNT(*)
    FROM customer c
    JOIN pay p ON c.cust_no = p.cust_no
    GROUP BY c.cust_no, name
);

2. Qual o nome dos empregados que processaram encomendas em todos os dias de 2022 em que houve encomendas?

In [ ]:
%%sql
SELECT DISTINCT name
FROM Employee e1
JOIN process USING (ssn)
JOIN pay p1 USING (order_no)
JOIN orders o1 USING (order_no)
WHERE NOT EXISTS (
    SELECT DISTICNT date
      FROM orders o2
      JOIN pay p2 USING(order_no)
      WHERE o1.date = o2.date
    EXCEPT
    SELECT DISTINCT date
      FROM orders
      WHERE date < '2023-01-01'
      AND date > '2021-12-31'
);


3. Quantas encomendas foram realizadas mas não pagas em cada mês de 2022?

In [ ]:
%%sql
SELECT 
  EXTRACT(MONTH FROM date) as Month, COUNT(*) as to_pay
FROM orders o
WHERE o.order_no NOT IN (
  SELECT order_no
  FROM pay
)
AND date < '2023-01-01'
AND date > '2021-12-31'
GROUP BY
  EXTRACT(MONTH FROM date)
;

## 3. Vistas

Crie uma vista que resuma as informações mais importantes sobre as vendas de produtos, combinando informações de diferentes tabelas do esquema de base de dados. A vista deve ter o seguinte esquema:

product_sales(sku, order_no, qty, total_price, year, month, day_of_month, day_of_week, city)

In [ ]:
%%sql
-- CREATE VIEW ...
CREATE VIEW product_sales AS
SELECT 
  c.SKU, 
  c.order_no, 
  c.qty, 
  c.qty * p.price AS total_price, 
  EXTRACT(YEAR FROM o.date) AS year,
  EXTRACT(MONTH FROM o.date) AS month,
  EXTRACT(DAY FROM o.date) AS day_of_month,
  EXTRACT(DOW FROM o.date) AS day_of_week,
  SUBSTRING(SUBSTRING(cst.address FROM '[0-9]{3} .+') FROM '[[:<:]][A-Z].+') AS city
  FROM contains c
  JOIN orders o USING (order_no)
  JOIN product p USING (SKU)
  JOIN customer cst USING (cust_no)
  ;


## 4. Desenvolvimento de Aplicação

### Explicação da arquitetura da aplicação web, incluindo um link para uma versão de trabalho e as relações entre os vários ficheiros na pasta web/arquivos

A app começa com uma página incial (index.html) com um botão que redireciona para a página de login. Na página de login (login_page.html) o utilizador ou entra como manager ou com uma das contas de customer.

Na parte da app do customer o utilizador apenas pode criar e pagar as orders correspondentes ao mesmo.(orders_index_private.html, order_payment, add_order.html)

Na parte da app do manager temos uma navbar com as opções: Customers, Orders, Products, Suppliers.(navbar.html)
Na parte dos customers é possivel visualizar todos os customer, apagar um deles ou criar um novo.(/customer)
Na parte das orders é possivel visualizar as orders de cada cliente e ver se foram pagas ou não.(order_customer.html, orders_index.html)
Na parte dos products é possivel visualizar todos os produtos e criar novos. Podemos editar ou visualizar todos os detalhes. Também é possivel apagar.(/products)
na parte dos suppliers é possivel visualizar todos os supliers e apagar ou criar novos. (/supplier)

Além disto implementamos também paginação, verificação de inputs, tentámos assegurar a segurança da app através da implementação de transações.

Para customizar a app utilizamos bootstrap.

## 5. Consultas OLAP

Usando a vista desenvolvida para a Questão 3, escreva duas consultas SQL que permitam analisar:

In [ ]:
-- Criação de uma tabela auxiliar para guardar todos os dias do ano de 2022
%%sql
-- SELECT ...
CREATE TABLE data AS
SELECT 
  DATE '2022-01-01' + SEQUENCE.DAY AS date
FROM 
  GENERATE_SERIES(0, 365) AS SEQUENCE(DAY);

1. As quantidade e valores totais de venda de cada produto em 2022, globalmente, por cidade, por mês, dia do mês e dia da semana

In [ ]:
%%sql
SELECT 
  p.name AS product_name,
  ps.city,
  EXTRACT(MONTH FROM d.date) AS month,
  EXTRACT(DAY FROM d.date) AS day_of_month,
  EXTRACT(DOW FROM d.date) AS day_of_week,
  SUM(ps.qty) AS total_quantity,
  SUM(ps.total_price) AS total_value
FROM 
  product_sales ps
JOIN 
  product p ON ps.SKU = p.SKU
RIGHT JOIN 
  data d ON (ps.year = EXTRACT(YEAR FROM d.date) AND ps.month = EXTRACT(MONTH FROM d.date) AND ps.day_of_month = EXTRACT(DAY FROM d.date))
WHERE 
  EXTRACT(YEAR FROM d.date) = 2022 AND TOTAL_PRICE IS NOT NULL
GROUP BY 
  GROUPING SETS ((p.name, ps.city, EXTRACT(MONTH FROM d.date), EXTRACT(DAY FROM d.date), EXTRACT(DOW FROM d.date)), ())
ORDER BY 
  p.name, ps.city, EXTRACT(MONTH FROM d.date), EXTRACT(DAY FROM d.date), EXTRACT(DOW FROM d.date);

2. O valor médio diário das vendas de todos os produtos em 2022, globalmente, por mês e dia da semana

In [ ]:
%%sql
SELECT 
  EXTRACT(MONTH FROM d.date) AS month,
  EXTRACT(DOW FROM d.date) AS day_of_week,
  AVG(ps.total_price) AS average_daily_sales
FROM 
  product_sales ps
RIGHT JOIN 
  data d ON (ps.year = EXTRACT(YEAR FROM d.date) AND ps.month = EXTRACT(MONTH FROM d.date) AND ps.day_of_month = EXTRACT(DAY FROM d.date))
WHERE 
  EXTRACT(YEAR FROM d.date) = 2022 AND ps.total_price IS NOT NULL
GROUP BY 
  GROUPING SETS ((EXTRACT(MONTH FROM d.date), EXTRACT(DOW FROM d.date)), ())
ORDER BY 
  EXTRACT(MONTH FROM d.date), EXTRACT(DOW FROM d.date);

## 6. Índices

Indique, com a devida justificação, que tipo de índice(s), sobre qual(is) atributo(s) e sobre qual(is) tabela(s) faria sentido criar, de forma a agilizar a execução de cada uma das seguintes consultas: 

### 6.1
SELECT order_no
FROM orders 
JOIN contains USING (order_no) 
JOIN product USING (SKU) 
WHERE price > 50 AND 
EXTRACT(YEAR FROM date) = 2023

### Tipo de Índice, Atributos & Justificação

O mais adequado para agilizar a execução desta consulta seria criar 2 Indices:

Um Indice simples na tabela product sobre o atributo price melhoraria o desempenho. Este seria um Índice do tipo B-tree pois neste caso estamos a fazer uma consulta de range e os Indices B-tree são ideais para isso mesmo.

O outro Indice simples na tabela orders sobre o atributo date melhoraria o desempenho. Este também seria um Índice do tipo B-tree. Neste caso a consulta não é uma consulta de range mas sim de intervalo (YEAR)  sendo também o indice do tipo B-tree o ideal para isso mesmo.

Poderiamos pensar em criar um indice composto na tabela contains com os atributos order_no e SKU para ajudar no JOIN , MAS como (order_no, SKU) é uma chave primária da tabela contains não é necessário criar o indice pois o indice já está associado automaticamente à chave primária assim evitamos a criação de um indice redundante. Portanto ficamo-nos pelo uso dos dois indices referidos acima.

### 6.2
SELECT order_no, SUM(qty*price)
FROM contains 
JOIN product USING (SKU) 
WHERE name LIKE ‘A%’ 
GROUP BY order_no;

### Tipo de Índice, Atributos & Justificação

O mais adequado para agilizar a execução desta consulta seria criar 2 Indices:

Um Índice simples na tabela product sobre o atributo name, melhoraria o desempenho.Este seria um Índice do tipo B-tree pois estamos a fazer uma correspondência parcial de strings (LIKE 'A%').

Um Índice simples na tabela contains sobre o atributo order_no, melhoraria o desempenho.Este seria um Índice do tipo B-tree que otimizaria o GROUP BY.

